In [ ]:
from segment_anything import sam_model_registry
from segment_anything import SamAutomaticMaskGenerator,SamPredictor
import nibabel as nib
import numpy as np
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import json
from PIL import Image
import cv2
from pycocotools import mask as mask_utils
import torch
import os
from torch.utils.data import DataLoader


device = "cuda"
# load default model
# Loading the model based on checkpoint
sam_model = sam_model_registry["vit_h"](checkpoint="./checkpoint/sam_vit_h_4b8939.pth")
sam_model.to(device=device)
predictor = SamPredictor(sam_model=sam_model)

In [ ]:
# Define dataset
import torch.nn.functional as F
from torchvision.transforms import Resize
from segment_anything import SamPredictor, sam_model_registry, utils

custom_dataset = "./data/dataset_0.json"
optimizer = torch.optim.Adam(sam_model.mask_decoder.parameters())

# Set up loss function
loss_fn = torch.nn.MSELoss()

# Load custom dataset
# Here, we should replace 'custom_dataset' with the dataset loader
# Define transforms
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
num_epochs = 2
for epoch in range(num_epochs):  # num_epochs to be defined
    for input_image, box_torch, gt_binary_mask in custom_dataset:
        input_image, box_torch, gt_binary_mask = input_image.to(device), box_torch.to(device), gt_binary_mask.to(device)

        # Image encoding
        with torch.no_grad():
            image_embedding = sam_model.image_encoder(input_image)
        
        # Prompt encoding
        with torch.no_grad():
            sparse_embeddings, dense_embeddings = sam_model.prompt_encoder(points=None, boxes=box_torch, masks=None)

        # Mask decoding
        low_res_masks, iou_predictions = sam_model.mask_decoder(
            image_embeddings=image_embedding,
            image_pe=sam_model.prompt_encoder.get_dense_pe(),
            sparse_prompt_embeddings=sparse_embeddings,
            dense_prompt_embeddings=dense_embeddings,
            multimask_output=False,
        )

        # Postprocessing
        upscaled_masks = sam_model.postprocess_masks(low_res_masks, input_size, original_image_size).to(device)

        # Generate binary mask
        binary_mask = F.normalize(F.threshold(upscaled_masks, 0.0, 0)).to(device)

        # Calculate loss and backpropagate
        loss = loss_fn(binary_mask, gt_binary_mask)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

# Save the fine-tuned model
torch.save(sam_model.state_dict(), 'fine_tuned_sam.pth')